# 千帆sdk调用一言插件

千帆sdk支持调用千帆插件，目前支持的插件有：
- 说图解画（ImageAI）：基于图片进行文字创作、回答问题，帮你写文案、想故事、图生图。暂仅支持10MB以内的图片。
- 览卷文档（ChatFile）：原ChatFile，可基于文档完成摘要、问答、创作等任务，仅支持10MB以内的文档，不支持扫描件。
- E言易图（eChart）：基于Apache ECharts为您提供数据洞察和图表制作，目前支持柱状图、折线图、饼图、雷达图、散点图、漏斗图、思维导图（树图）。

本文以调用PluginV2插件为例，qianfan版本>=0.3.0，介绍三种插件的效果

## 说图解画（ImageAI）
插件ImageAI能根据图片进行图生文、图生图操作
由于输入文件必须符合格式要求，所以此处封装了query_url函数，将图片url转化为符合要求的格式
目前支持的格式有：jpg, jpeg, png, bmp, webp
url中必须包含文件后缀名，并且可下载，否则无法识别。可以使用wget命令验证下载是否成功

In [1]:
from qianfan import Plugin
plug = Plugin(model="EBPluginV2")

def query_url(name, url, query=None, type='img'):
    if type == 'img' and url.rsplit('.',1)[-1] not in ['jpg', 'jpeg', 'png', 'bmp', 'webp']:
        raise ValueError('url must contain file suffix')
    elif type == 'file' and url.rsplit('.',1)[-1] not in ['doc','docx','pdf']:
        raise ValueError('url must contain file suffix')
    elif type not in ['img', 'file']:
        raise ValueError('only img or file type supported')
    
    url_query = f'<{type}>{name}</{type}><url>{url}</url> {query}'
    return url_query

In [2]:
from qianfan import QfMessages

query = query_url('web picture', 'https://qianfan-doc.bj.bcebos.com/imageai/cow2.jpeg', "图片上画了什么?",)

msg = QfMessages()
msg.append(query)

resp = plug.do(msg, plugins=['ImageAI'], stream=False)
print(resp['result'])

[INFO] [02-27 17:30:15] openapi_requestor.py:244 [t:8603288192]: requesting llm api endpoint: /erniebot/plugin
[INFO] [02-27 17:30:15] oauth.py:207 [t:8603288192]: trying to refresh access_token for ak `rRlk1M***`
[INFO] [02-27 17:30:16] oauth.py:220 [t:8603288192]: sucessfully refresh access_token


图片上画了一头黑白色奶牛，这头奶牛正在看着摄像机。背景是蓝色的天空。


## 览卷文档（ChatFile）
插件ChatFile能基于文档完成摘要、问答、创作等任务，仅支持10MB以内的文档，不支持扫描件。
目前支持的格式有：doc, docx, pdf

In [3]:
query = query_url('浅谈牛奶的营养与消费趋势','https://qianfan-doc.bj.bcebos.com/chatfile/%e6%b5%85%e8%b0%88%e7%89%9b%e5%a5%b6%e7%9a%84%e8%90%a5%e5%85%bb%e4%b8%8e%e6%b6%88%e8%b4%b9%e8%b6%8b%e5%8a%bf.docx', "请问牛奶的成分有哪些？", 'file')
msg = QfMessages()
msg.append(query)

resp = plug.do(msg, plugins=['ChatFile'], stream=False)
print(resp['result'])

[INFO] [02-27 17:30:20] openapi_requestor.py:244 [t:8603288192]: requesting llm api endpoint: /erniebot/plugin


牛奶是一种营养丰富的食品，其成分种类繁多，各具特色。以下是牛奶的主要成分及其简要介绍：

1. 水分：牛奶中含有大约87%的水分，这些水分对于维持人体的正常生理功能至关重要。
2. 脂肪：牛奶中的脂肪以乳脂的形式存在，其含量因牛的品种、饲料、产奶阶段等因素而异。一般来说，全脂牛奶的脂肪含量约为3%-4%，而低脂或脱脂牛奶则经过加工处理，去除了部分或全部脂肪。
3. 蛋白质：牛奶中含有丰富的蛋白质，包括酪蛋白和乳清蛋白等。这些蛋白质对于人体的生长发育、组织修复以及免疫功能等方面都具有重要作用。
4. 乳糖：乳糖是牛奶中的主要碳水化合物成分，它是一种双糖，由葡萄糖和半乳糖组成。乳糖在人体内可被分解为单糖并被吸收利用，为人体提供能量。
5. 矿物质：牛奶中含有多种矿物质，如钙、磷、钾、镁等。这些矿物质对于维持人体的骨骼健康、神经传导、心肌收缩等方面都具有重要作用。
6. 维生素：牛奶中还含有多种维生素，如维生素A、D、B2、B12等。这些维生素对于维持人体的正常生理功能、促进新陈代谢等方面都具有重要作用。

此外，牛奶中还含有一些生物活性成分，如免疫球蛋白、生长因子等，这些成分对人体健康也有益处。总的来说，牛奶是一种营养丰富的食品，适量饮用可以为人体提供多种营养成分，有助于维持身体健康。


## E言易图（eChart）
插件eChart能根据输入的文本生成Apache ECharts图表的配置代码，用户只需将生成的代码复制到ECharts js代码的options字典中，即可生成图表。

In [4]:
query = "帮我画一个饼状图：8月的用户反馈中，BUG有100条，需求有100条，使用咨询100条，总共300条反馈。"
msg = QfMessages()
msg.append(query)

resp = plug.do(msg, plugins=['eChart'], stream=False)
print(resp['result'])

[INFO] [02-27 17:30:46] openapi_requestor.py:244 [t:8603288192]: requesting llm api endpoint: /erniebot/plugin




```echarts-option
[{"series":[{"type":"pie","name":"反馈数量","data":[{"name":"BUG","value":100},{"name":"需求","value":100},{"name":"使用咨询","value":100},{"name":"总计","value":300}],"label":{"show":true,"formatter":"{b}：{c} 条"}}],"title":{"text":"8月用户反馈类型分布（反馈数量）"},"tooltip":{"show":true},"legend":{"show":true,"bottom":15}}]
```



**图表数据:**

| 反馈类型 | 反馈数量 |
| :--: | :--: |
| BUG | 100 |
| 需求 | 100 |
| 使用咨询 | 100 |
| 总计 | 300 |

我（文心一言）是百度开发的人工智能模型，通过分析大量公开文本信息进行学习。然而，我所提供的信息可能存在误差。因此上文内容仅供参考，并不应被视为专业建议。
